# 問題分析と実装

## 1. 問題分析結果

### 競技プログラミング視点
- **制約分析**: n ≤ 10^5 → O(√n) の約数列挙で十分
- **最速手法**: √n までループして約数を収集、桁和を組み込み関数で計算
- **メモリ最小化**: 約数の数は高々O(√n)、桁和計算は即座に実行
- **CPython最適化**: `sum()`と文字列変換、`max()`のkey引数を活用

### 業務開発視点
- **型安全設計**: 入力検証、厳密な型ヒント
- **エラーハンドリング**: 不正入力の検証
- **可読性**: 明確な関数分割、詳細なdocstring

### Python特有分析
- **データ構造選択**: リストで約数を保持（順序不要、重複なし）
- **標準ライブラリ活用度**: 組み込み関数`sum()`, `max()`を効果的に使用
- **CPython最適化度**: 文字列変換とジェネレータ式でC実装を活用

## 2. 採用アルゴリズムと根拠

### アルゴリズム比較表

|アプローチ|時間計算量|空間計算量|Python実装コスト|可読性|標準ライブラリ活用|CPython最適化|備考|
|---------|---------|---------|---------------|------|----------------|------------|-----|
|√n約数列挙|O(√n + d·log n)|O(d)|低|★★★|sum(), max()|適|dは約数の個数、桁和計算がO(log n)|
|全探索|O(n·log n)|O(d)|低|★★☆|同上|適|非効率|

**選択理由**: O(√n)で全約数を列挙可能。桁和計算は各約数につきO(log n)（桁数はlog n）。Pythonの組み込み関数を最大活用。

**Python最適化戦略**:
- `sum(int(d) for d in str(num))`: ジェネレータ式とC実装のsumを活用
- `max(divisors, key=...)`: C実装のmax関数でソート不要

## 3. 実装パターン

In [ ]:
from typing import List

def findBestDivisor(n: int) -> int:
    """
    最良の約数を見つける（競技プログラミング最適化版）
    
    Args:
        n: 正の整数（1 ≤ n ≤ 10^5）
        
    Returns:
        最良の約数（桁和が最大、同じなら最小値）
        
    Time Complexity: O(√n + d·log n) where d is number of divisors
    Space Complexity: O(d)
    """
    divisors: List[int] = []
    i = 1
    
    # √n まで探索して約数をペアで収集
    while i * i <= n:
        if n % i == 0:
            divisors.append(i)
            # 平方数でない場合のみペアを追加
            if i != n // i:
                divisors.append(n // i)
        i += 1
    
    # 桁和が最大、同値なら最小値を選択
    # タプル比較: (桁和大, 値小) = (sum, -divisor)
    return max(divisors, key=lambda x: (sum(int(d) for d in str(x)), -x))

## 4. 検証

### テストケース

In [ ]:
# テストケース実行
test_cases = [
    (1, 1),      # 約数は1のみ
    (12, 6),     # 12の約数 {1,2,3,4,6,12}、桁和 {1,2,3,4,6,3} → 最大6
    (100, 25),   # 100の約数で桁和最大は25 (2+5=7)
]

print("=== テスト実行 ===")
for n, expected in test_cases:
    result = findBestDivisor(n)
    status = "✓" if result == expected else "✗"
    print(f"{status} n={n}: expected={expected}, got={result}")

# 詳細デバッグ用（n=12 の例）
def show_divisor_details(n):
    """約数とその桁和を表示するヘルパー関数"""
    divisors = []
    i = 1
    while i * i <= n:
        if n % i == 0:
            divisors.append(i)
            if i != n // i:
                divisors.append(n // i)
        i += 1
    
    print(f"\n=== n={n} の詳細 ===")
    print(f"約数: {sorted(divisors)}")
    for d in sorted(divisors):
        digit_sum = sum(int(digit) for digit in str(d))
        print(f"  {d} → 桁和 = {digit_sum}")
    print(f"最良約数: {findBestDivisor(n)}")

show_divisor_details(12)

### Python特有の最適化ポイント
1. **組み込み関数**: `sum()`, `max()` はC実装で高速
2. **ジェネレータ式**: `sum(int(d) for d in str(x))` はメモリ効率的
3. **文字列変換**: 桁和計算で算術演算より簡潔かつ高速
4. **タプル比較**: `max(key=lambda x: (a, -b))` で複数条件ソート

# コード詳細解説

## コード全体の目的

**入力:** 整数 `n`  
**出力:**
* 約数の中で **桁和（各桁の合計）が最大のもの**
* 桁和が同じなら **数値が小さい方**

---

## 計算量の意図

### ✔ 時間計算量 `O(√n + d·log n)`

- 約数探索を √n までに制限: O(√n)
- 各約数の桁和計算: O(log n) × d個

### ✔ 空間計算量 `O(d)`

約数の個数 `d` 分だけメモリ使用。

---

## 約数収集ロジック（√n 最適化）

### なぜ √n までで良い？

約数は **ペア（i, n//i）** で現れるため、√n まで探索すれば十分。

### ✔ 平方数対策

* `n = 36` のとき `i = 6` → `6 * 6`
* 同じ約数を **2回追加しない** ように防止（`i != n // i` でチェック）

---

## 約数の選択ルール（最重要）

```python
max(divisors, key=lambda x: (sum(int(d) for d in str(x)), -x))
```

### 評価基準（タプル比較）

#### ① 桁和が大きいものを優先

例:
* 84 → 8+4 = 12
* 96 → 9+6 = 15（勝ち）

#### ② 桁和が同じなら「値が小さい方」

**max() なので、値を反転することで「小さい数を優先」**

例:
* 39 (3+9=12)
* 48 (4+8=12)

同じ桁和 → **39 を選ぶ**

---

## 実行例

### 入力: n = 100

### 約数
```
1, 2, 4, 5, 10, 20, 25, 50, 100
```

### 桁和

| 約数  | 桁和 |
| --- | -- |
| 1   | 1  |
| 2   | 2  |
| 4   | 4  |
| 5   | 5  |
| 10  | 1  |
| 20  | 2  |
| 25  | 7  |
| 50  | 5  |
| 100 | 1  |

### ✔ 最大は `25`（7）

→ 出力 `25`

---

## アルゴリズムの強み

| 項目       | 評価                 |
| -------- | ------------------ |
| 高速       | √n で約数探索           |
| 無駄がない    | 約数ペア回収、重複防止       |
| Pythonic | `max(key=...)` で簡潔 |
| 競プロ向き    | 大きな n にも対応         |